# Apply the constrained setup and run in openscm-runner

- Working Group 3 style setup
- TODO: parallel
- TODO: update variable names for openscm-runner-0.5.0

Theme Song: The Chain<br>
Artist: Fleetwood Mac<br>
Album: Rumours<br>
Released: 1977

In [1]:
import os.path

import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pyam
from fair.forward import fair_scm
from scmdata import ScmRun, run_append
from tqdm import tqdm_notebook

import openscm_runner
from openscm_runner.run import run
from openscm_runner.adapters import FAIR

<IPython.core.display.Javascript object>

/nfs/b0110/Users/mencsm/miniconda3/envs/ar6/lib/python3.7/site-packages/openscm_runner/progress.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
openscm_runner.__version__

'0.1.0-alpha.1+309.g0ab51a4'

In [3]:
fair = FAIR()
fair.get_version()

'1.6.2'

In [4]:
with open('../data_output_large/fair-samples/fair-1.6.2-wg3-params.json') as f:
    config_list = json.load(f)

In [5]:
# what a pain

species = [
'Emissions|BC',
'Emissions|CH4',
'Emissions|CO',
'Emissions|CO2|MAGICC AFOLU',
'Emissions|CO2|MAGICC Fossil and Industrial',
'Emissions|F-Gases|HFC|HFC125',
'Emissions|F-Gases|HFC|HFC134a',
'Emissions|F-Gases|HFC|HFC143a',
'Emissions|F-Gases|HFC|HFC227ea',
'Emissions|F-Gases|HFC|HFC23',
'Emissions|F-Gases|HFC|HFC245fa',
'Emissions|F-Gases|HFC|HFC32',
'Emissions|F-Gases|HFC|HFC4310mee',
'Emissions|F-Gases|PFC|C2F6',
'Emissions|F-Gases|PFC|C6F14',
'Emissions|F-Gases|PFC|CF4',
'Emissions|F-Gases|SF6',
'Emissions|Montreal Gases|CCl4',
'Emissions|Montreal Gases|CFC|CFC11',
'Emissions|Montreal Gases|CFC|CFC113',
'Emissions|Montreal Gases|CFC|CFC114',
'Emissions|Montreal Gases|CFC|CFC115',
'Emissions|Montreal Gases|CFC|CFC12',
'Emissions|Montreal Gases|CH3Br',
'Emissions|Montreal Gases|CH3CCl3',
'Emissions|Montreal Gases|CH3Cl',
'Emissions|Montreal Gases|HCFC141b',
'Emissions|Montreal Gases|HCFC142b',
'Emissions|Montreal Gases|HCFC22',
'Emissions|Montreal Gases|Halon1202',
'Emissions|Montreal Gases|Halon1211',
'Emissions|Montreal Gases|Halon1301',
'Emissions|Montreal Gases|Halon2402',
'Emissions|N2O',
'Emissions|NH3',
'Emissions|NOx',
'Emissions|OC',
'Emissions|Sulfur',
'Emissions|VOC']

In [6]:
df_fair = ScmRun('../data_input_large/rcmip-emissions-annual-means-v5-1-0.csv', lowercase_cols=True)
df_fair.filter(
    scenario=['ssp119','ssp126','ssp245','ssp370','ssp434','ssp460','ssp534-over','ssp585'], 
#    year=range(2015,2301),
    year=range(2015,2101),
    variable=species,
    region='World', 
    inplace=True
)
print(len(df_fair))
#df_fair.head(50)

312


In [7]:
nt = df_fair.time_points.years()[-1] - 1750 + 1
nt

351

In [8]:
# # add in natural emissions and natural forcing
# ch4_n2o_df = pd.read_csv('../data/cmip6_natural_ch4_n2o.csv')
# ch4_n2o = ch4_n2o_df.values[:nt,1:]
# sol_vol_df = pd.read_csv('../data/cmip6_natural_forcing.csv')
# solar_forcing = sol_vol_df.values[:nt,1]
# volcanic_forcing = sol_vol_df.values[:nt,2]

In [9]:
#config_list[0]

In [10]:
# convert lists to numpy arrays
updated_config = []
for i in range(len(config_list)):
    updated_config.append({})
    for key, value in config_list[i].items():
        if type(value)==list:
            updated_config[i][key] = np.asarray(value)
        else:
            updated_config[i][key] = value
#     updated_config[i]['endyear'] = 2300
#     updated_config[i]['natural'] = ch4_n2o
#     updated_config[i]['F_solar'] = solar_forcing
#     updated_config[i]['F_volcanic'] = volcanic_forcing

In [11]:
# need parallel FaIR in openscm-runner

x = run(
    climate_models_cfgs={
        "FAIR": updated_config,
    },
    scenarios=df_fair,
    output_variables=(
        "Surface Temperature",
        "Surface Temperature (GMST)",
        "Atmospheric Concentrations|CO2",
        "Atmospheric Concentrations|CH4",
        "Atmospheric Concentrations|N2O",
        "Effective Radiative Forcing",
        "Effective Radiative Forcing|CO2",
        "Effective Radiative Forcing|CH4",
        "Effective Radiative Forcing|N2O",
        "Effective Radiative Forcing|Greenhouse Gases",
        "Effective Radiative Forcing|Aerosols",
        "Effective Radiative Forcing|Aerosols|Direct Effect|BC",
        "Effective Radiative Forcing|Aerosols|Direct Effect|OC",
        "Effective Radiative Forcing|Aerosols|Direct Effect|Sulfur",
        "Effective Radiative Forcing|Aerosols|Direct Effect|Nitrate",
        "Effective Radiative Forcing|Aerosols|Direct Effect",
        "Effective Radiative Forcing|Aerosols|Indirect Effect",
        "Heat Uptake",
        "Heat Uptake|Ocean",
        "Airborne Fraction",
    ),
)

Climate models:   0%|          | 0.00/1.00 [00:00<?, ?it/s]

Creating FaIR emissions inputs:   0%|          | 0.00/8.00 [00:00<?, ?it/s]

Surface Temperature not available from FaIR
Surface Temperature (GMST) not available from FaIR
Effective Radiative Forcing|Aerosols|Direct Effect|Sulfur not available from FaIR
Surface Temperature not available from FaIR
Surface Temperature not available from FaIR
Surface Temperature (GMST) not available from FaIR
Surface Temperature (GMST) not available from FaIR
Effective Radiative Forcing|Aerosols|Direct Effect|Sulfur not available from FaIR
Effective Radiative Forcing|Aerosols|Direct Effect|Sulfur not available from FaIR
Surface Temperature not available from FaIR
Surface Temperature (GMST) not available from FaIR
Effective Radiative Forcing|Aerosols|Direct Effect|Sulfur not available from FaIR
Surface Temperature not available from FaIR
Surface Temperature (GMST) not available from FaIR
Effective Radiative Forcing|Aerosols|Direct Effect|Sulfur not available from FaIR
Surface Temperature not available from FaIR
Surface Temperature (GMST) not available from FaIR
Effective Radiative 

Surface Temperature not available from FaIR
Effective Radiative Forcing|Aerosols|Direct Effect|Sulfur not available from FaIR
Surface Temperature (GMST) not available from FaIR
Surface Temperature not available from FaIR
Surface Temperature (GMST) not available from FaIR
Surface Temperature not available from FaIR
Surface Temperature not available from FaIR
Surface Temperature (GMST) not available from FaIR
Surface Temperature (GMST) not available from FaIR
Effective Radiative Forcing|Aerosols|Direct Effect|Sulfur not available from FaIR
Effective Radiative Forcing|Aerosols|Direct Effect|Sulfur not available from FaIR
Effective Radiative Forcing|Aerosols|Direct Effect|Sulfur not available from FaIR
Effective Radiative Forcing|Aerosols|Direct Effect|Sulfur not available from FaIR
Surface Temperature not available from FaIR
Surface Temperature not available from FaIR
Surface Temperature not available from FaIR
Surface Temperature (GMST) not available from FaIR
Surface Temperature not ava

Effective Radiative Forcing|Aerosols|Direct Effect|Sulfur not available from FaIR
Surface Temperature (GMST) not available from FaIR
Surface Temperature (GMST) not available from FaIR
Surface Temperature (GMST) not available from FaIR
Effective Radiative Forcing|Aerosols|Direct Effect|Sulfur not available from FaIR
Effective Radiative Forcing|Aerosols|Direct Effect|Sulfur not available from FaIR
Effective Radiative Forcing|Aerosols|Direct Effect|Sulfur not available from FaIR
Surface Temperature not available from FaIR
Surface Temperature not available from FaIR
Surface Temperature (GMST) not available from FaIR
Surface Temperature not available from FaIR
Surface Temperature (GMST) not available from FaIR
Surface Temperature (GMST) not available from FaIR
Effective Radiative Forcing|Aerosols|Direct Effect|Sulfur not available from FaIR
Effective Radiative Forcing|Aerosols|Direct Effect|Sulfur not available from FaIR
Surface Temperature not available from FaIR
Surface Temperature not av

In [ ]:
# convert to ScmRun for better plotting functionality
x = ScmRun(x.timeseries())

In [ ]:
x.tail()

In [ ]:
def new_timeseries(
    n=100,
    count=1,
    model="example",
    scenario="ssp119",
    variable="Surface Temperature",
    unit="K",
    region="World",
    cls=ScmRun,
    **kwargs,
):
    data = np.random.rand(n, count) * np.arange(n)[:, np.newaxis]
    index = 2000 + np.arange(n)
    return cls(
        data,
        columns={
            "model": model,
            "scenario": scenario,
            "variable": variable,
            "region": region,
            "unit": unit,
            **kwargs,
        },
        index=index,
    )

In [ ]:
# save output to netCDF
x.to_nc('../data_output_large/fair-samples/ssp_emissions_driven_runs.nc', dimensions=["run_id", "scenario", "model"])

In [ ]:
x = ScmRun.from_nc('../data_output_large/fair-samples/ssp_emissions_driven_runs.nc')

from scmdata.plotting import RCMIP_SCENARIO_COLOURS

In [ ]:
plt.rcParams['font.size'] = 14
ax = plt.figure(figsize=(12, 7)).add_subplot(111)
# x.filter(variable="Surface Temperature").relative_to_ref_period_mean(year=range(1985,2005)).lineplot(
#     hue="scenario", style="model", ax=ax, time_axis="year", palette=RCMIP_SCENARIO_COLOURS,
#     hue_order=RCMIP_SCENARIO_COLOURS.keys(),
# )
x.filter(variable="Surface Temperature").relative_to_ref_period_mean(year=range(1850,1900)).lineplot(
    hue="scenario",
    ax=ax,
    time_axis="year",
    palette=RCMIP_SCENARIO_COLOURS,
    hue_order=RCMIP_SCENARIO_COLOURS.keys(),
)
ax.set_xlim(1850,2100)
#ax.axhline(0, color='k', ls=':')
ax.grid()
ax.set_title('SSP projections with FaIR: 1850-2100')
plt.tight_layout()

In [ ]:
ax = plt.figure(figsize=(12, 7)).add_subplot(111)
x.filter(variable="Effective Radiative Forcing|Aerosols").lineplot(
    hue="scenario", style="model", ax=ax, time_axis="year"
)

In [ ]:
ax = plt.figure(figsize=(12, 7)).add_subplot(111)
x.filter(variable="Effective Radiative Forcing|Aerosols|Direct Effect|BC").lineplot(
    hue="scenario", style="model", ax=ax, time_axis="year"
)

In [ ]:
ax = plt.figure(figsize=(12, 7)).add_subplot(111)
x.filter(variable="Effective Radiative Forcing|Aerosols|Direct Effect|OC").lineplot(
    hue="scenario", style="model", ax=ax, time_axis="year"
)

In [ ]:
ax = plt.figure(figsize=(12, 7)).add_subplot(111)
x.filter(variable="Effective Radiative Forcing|Aerosols|Direct Effect|Sulfur").lineplot(
    hue="scenario", style="model", ax=ax, time_axis="year"
)

In [ ]:
ax = plt.figure(figsize=(12, 7)).add_subplot(111)
x.filter(variable="Effective Radiative Forcing|Aerosols|Direct Effect|Nitrate").lineplot(
    hue="scenario", style="model", ax=ax, time_axis="year"
)

In [ ]:
ax = plt.figure(figsize=(12, 7)).add_subplot(111)
x.filter(variable="Effective Radiative Forcing|Aerosols|Direct Effect").lineplot(
    hue="scenario", style="model", ax=ax, time_axis="year"
)

In [ ]:
ax = plt.figure(figsize=(12, 7)).add_subplot(111)
x.filter(variable="Effective Radiative Forcing|Aerosols|Indirect Effect").lineplot(
    hue="scenario", style="model", ax=ax, time_axis="year"
)

In [ ]:
ax = plt.figure(figsize=(12, 7)).add_subplot(111)
x.filter(variable="Atmospheric Concentrations|CO2").lineplot(
    hue="scenario", style="model", ax=ax, time_axis="year"
)

In [ ]:
ax = plt.figure(figsize=(12, 7)).add_subplot(111)
x.filter(variable="Atmospheric Concentrations|CH4").lineplot(
    hue="scenario", style="model", ax=ax, time_axis="year"
)

In [ ]:
ax = plt.figure(figsize=(12, 7)).add_subplot(111)
x.filter(variable="Atmospheric Concentrations|N2O").lineplot(
    hue="scenario", style="model", ax=ax, time_axis="year"
)

In [ ]:
ax = plt.figure(figsize=(12, 7)).add_subplot(111)
x.filter(variable="Heat Uptake").lineplot(
    hue="scenario", style="model", ax=ax, time_axis="year"
)

In [ ]:
ax = plt.figure(figsize=(12, 7)).add_subplot(111)
x.filter(variable="Airborne Fraction").lineplot(
    hue="scenario", style="model", ax=ax, time_axis="year"
)

In [ ]:
x.filter(scenario='ssp119', variable="Atmospheric Concentrations|CO2").timeseries()

In [ ]:
x.filter(variable="Atmospheric Concentrations|N2O", scenario='ssp119', run_id=0).timeseries().values.squeeze()[264]

In [ ]:
x.filter(variable="Atmospheric Concentrations|CH4", scenario='ssp119', run_id=0).timeseries().values.squeeze()[264]